Inserting necessary libraries



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tensorflow.__version__

'2.9.2'

In [ ]:
tensorflow.keras.__version__

'2.9.0'

Image Data Augumentation

In [ ]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

Loading our data and performing Data Augumentation

In [ ]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'/content/drive/MyDrive/dataset/train_set',target_size=(64, 64),batch_size=24,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'/content/drive/MyDrive/dataset/test_set',target_size=(64, 64),batch_size=24,
                                            color_mode='rgb',class_mode='categorical')

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [ ]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [ ]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [ ]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

Creating the Model

In [ ]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and poolingo
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2

In [ ]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 32)        9

Compiling the Model

In [ ]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Fitting the Model

In [ ]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=10, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/10
31/31 [==============================] - 41s 1s/step - loss: 1.2733 - accuracy: 0.3706 - val_loss: 1.3401 - val_accuracy: 0.2172
Epoch 2/10
31/31 [==============================] - 41s 1s/step - loss: 1.1257 - accuracy: 0.4811 - val_loss: 1.1353 - val_accuracy: 0.5354
Epoch 3/10
31/31 [==============================] - 39s 1s/step - loss: 0.8975 - accuracy: 0.6280 - val_loss: 1.1651 - val_accuracy: 0.4747
Epoch 4/10
31/31 [==============================] - 39s 1s/step - loss: 0.7673 - accuracy: 0.6914 - val_loss: 0.9801 - val_accuracy: 0.6869
Epoch 5/10
31/31 [==============================] - 39s 1s/step - loss: 0.6464 - accuracy: 0.7493 - val_loss: 0.8208 - val_accuracy: 0.7121
Epoch 6/10
31/31 [==============================] - 39s 1s/step - loss: 0.6411 - accuracy: 0.7547 - val_loss: 0.8856 - val_accuracy: 0.6515
Epoch 7/10
31/31 [==============================] - 39s 1s/step - loss: 0.5887 - accuracy: 0.7871 - val_loss: 0.7549 - val_accuracy: 0.7424
Epoch 8/10
31/31 [==

Saving the Model

In [ ]:
# Save the model
classifier.save('disaster.h5')

In [ ]:
!tar -zcvf disaster-classification.tgz disaster.h5

disaster.h5


IBM Deployment

In [ ]:
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 538 kB 4.8 MB/s 
     |████████████████████████████████| 132 kB 67.1 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 9.9 MB 62.7 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 140 kB 53.1 MB/s 
     |████████████████████████████████| 956 kB 57.5 MB/s 
     |████████████████████████████████| 135 kB 51.0 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.12.0-py3-none-any.whl size=73929 sha256=0c4f87ca37fbf8f265c46ce9bac457c809989290447c38e21e04736d1b6a158f
  Stored in directory: /root/.cache/pip/wheels/ec/94/29/2b57327cf00664b6614304f7958abd29d77ea0e5bbece2ea57
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.12.0-py3-none-any.whl size=562962 sha256=11c18b94c16

In [ ]:
!pip install ibm_watson_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 51 kB 659 kB/s 
     |████████████████████████████████| 824 kB 49.8 MB/s 
     |████████████████████████████████| 133 kB 44.4 MB/s 
     |████████████████████████████████| 547 kB 64.3 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72564 sha256=3d71165c3675fb72f7213e8f64126d0564d8fa9d59d6a000083cba40888ebea6
  Stored in directory: /root/.cache/pip/wheels/47/22/bf/e1154ff0f5de93cc477acd0ca69abfbb8b799c5b28a66b44c2
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501011 sha256=8d07c6e5acf1f2c49166e27f293c40528aafdca2c20484132303658e283106f4
  Stored in directory: /root/.cache/pip/wheels/6c/a2/e4/c16d02f809a3ea998e17cfd02c13369281f3d232aaf5902c19
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials ={
    "url":"https://eu-gb.ml.cloud.ibm.com",
    "apikey":"svC-EUm9-gSj4fy_tJwXdb-AsdXf-8GPIOt_DG2na3B4"
}

client = APIClient(wml_credentials)
client

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [ ]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:eu-gb:a/1b4158453a4e40e4b5e452cc930d8653:8b30ac71-a0a5-47ed-ba84-53eafca26d14::',
      'guid': '8b30ac71-a0a5-47ed-ba84-53eafca26d14',
      'name': 'Watson Machine Learning-u5',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'M_deploy',
    'scope': {'bss_account_id': '1b4158453a4e40e4b5e452cc930d8653'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '001ca5ec-7f2c-479b-8f96-de531248df78',
      'bucket_region': 'eu-gb-standard',
      'credentials': {'admin': {'access_key_id': 'e24101ccd12d471581773d49aac0ef49',
        'api_key': 'di__di7i9nWPVbomITQBdN_6Pad7-U2xHSLRkU1xzo4z',
        'secret_access_key': '16390a632088a029b23904ace89b27540744523250705774',
        'service_id': 'ServiceId-fefb3fd4-f0b3-4f99-acb0-e273e9c91726'},
       'editor': {'access_key_id': '0fe04e1d86ef46589edec6f6a2421041',
        'api_key'

In [ ]:
def guid_space_name(client,M_deploy):
  space = client.spaces.get_details()
  return(next(item for item in space['resources'] if item['entity']['name']==M_deploy)['metadata']['id'])

In [ ]:
space_uid = guid_space_name(client,'M_deploy')
space_uid

'32e4c41f-866e-49bf-a8d1-48bfbfb34b21'

In [ ]:
client.set.default_space(space_uid)

'SUCCESS'

In [ ]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [ ]:
software_space_uid = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [ ]:
model_details = client.repository.store_model(model='disaster-classification.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"Disaster Model",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [ ]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-10T05:54:45.584Z',
  'id': '21c7066c-fa0e-4621-b4cb-623ef40335b1',
  'modified_at': '2022-11-10T05:54:48.763Z',
  'name': 'Disaster Model',
  'owner': 'IBMid-662003YT1F',
  'resource_key': 'af2a202e-f97d-432c-8bdf-57d98d1265b4',
  'space_id': '32e4c41f-866e-49bf-a8d1-48bfbfb34b21'},
 'system': {'warnings': []}}

In [ ]:
model_id = client.repository.get_model_id(model_details)
model_id

'21c7066c-fa0e-4621-b4cb-623ef40335b1'

In [ ]:
client.repository.download(model_id,'Disaster_IBM_model.tar.gb')

Successfully saved model content to file: 'Disaster_IBM_model.tar.gb'


'/content/Disaster_IBM_model.tar.gb'